In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://hoopshype.com/salaries/players/'
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://hoopshype.com/salaries/players/2021-2022',
 'https://hoopshype.com/salaries/players/2020-2021',
 'https://hoopshype.com/salaries/players/2019-2020',
 'https://hoopshype.com/salaries/players/2018-2019',
 'https://hoopshype.com/salaries/players/2017-2018']

In [4]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:

    # provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Make python pause for some time
    sleep(2)

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table', class_='hh-salaries-ranking-table hh-salaries-table-sortable responsive')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('td')[1:4]:
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Create a dataframe using the headers as column names
    p_salary = pd.DataFrame(columns = headers) 
    
    # Create a for loop to fill p_salary
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "").strip() for j in row_data] 
        length = len(p_salary)
        p_salary.loc[length] = row

    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(p_salary)


Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Completed: 11 webpage(s) scraped.
Completed: 12 webpage(s) scraped.
Completed: 13 webpage(s) scraped.
Completed: 14 webpage(s) scraped.
Completed: 15 webpage(s) scraped.
Completed: 16 webpage(s) scraped.
Completed: 17 webpage(s) scraped.
Completed: 18 webpage(s) scraped.
Completed: 19 webpage(s) scraped.
Completed: 20 webpage(s) scraped.
Completed: 21 webpage(s) scraped.
Completed: 22 webpage(s) scraped.
Completed: 23 webpage(s) scraped.
Completed: 24 webpage(s) scraped.
Completed: 25 webpage(s) scraped.
Completed: 26 webpage(s) scraped.


In [5]:
dataframe_list

[                Player      2021/22   2021/22(*)
 0        Stephen Curry  $45,780,966  $45,780,966
 1         James Harden  $44,310,840  $44,310,840
 2            John Wall  $44,310,840  $44,310,840
 3    Russell Westbrook  $44,211,146  $44,211,146
 4         Kevin Durant  $42,018,900  $42,018,900
 ..                 ...          ...          ...
 648     Anthony Barber      $53,176      $53,176
 649       Luca Vildoza      $42,789      $42,789
 650     Zavier Simpson      $37,223      $37,223
 651  Mfiondu Kabengele      $19,186      $19,186
 652     Melvin Frazier      $13,294      $13,294
 
 [653 rows x 3 columns],
                 Player      2020/21   2020/21(*)
 0        Stephen Curry  $43,006,362  $45,325,029
 1    Russell Westbrook  $41,358,814  $43,588,654
 2           Chris Paul  $41,358,814  $43,588,654
 3            John Wall  $41,254,920  $43,479,158
 4         James Harden  $41,254,920  $43,479,158
 ..                 ...          ...          ...
 573       Cam Reynolds

In [97]:
# Merge the dataframes
from functools import reduce
test = reduce(lambda l, r: pd.merge(l, r, on='Player', how='outer'), dataframe_list)
test

,Player,2021/22,2021/22(*),2020/21,2020/21(*),2019/20,2019/20(*),2018/19,2018/19(*),2017/18,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Stephen Curry,"$45,780,966","$45,780,966","$43,006,362","$45,325,029","$40,231,758","$42,674,629","$37,457,154","$40,386,520","$34,682,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$30,570,000","$32,960,751","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$19,169,800","$20,668,989","$18,063,850",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$44,211,146","$41,358,814","$43,588,654","$38,506,482","$40,844,595","$35,665,000","$38,454,209","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,018,900","$40,108,950","$42,271,404","$37,199,000","$39,457,722","$30,000,000","$32,346,173","$25,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$220,000","$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$205,200","$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$190,100","$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$184,100","$319,203"


In [98]:
# Save file, jic
test.to_csv('nba_salaries_uncleaned.csv', encoding='utf-8', index=False)

In [99]:
nba_sal = pd.read_csv('nba_salaries_uncleaned.csv', encoding='utf-8')
nba_sal

,Player,2021/22,2021/22(*),2020/21,2020/21(*),2019/20,2019/20(*),2018/19,2018/19(*),2017/18,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Stephen Curry,"$45,780,966","$45,780,966","$43,006,362","$45,325,029","$40,231,758","$42,674,629","$37,457,154","$40,386,520","$34,682,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$30,570,000","$32,960,751","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$19,169,800","$20,668,989","$18,063,850",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$44,211,146","$41,358,814","$43,588,654","$38,506,482","$40,844,595","$35,665,000","$38,454,209","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,018,900","$40,108,950","$42,271,404","$37,199,000","$39,457,722","$30,000,000","$32,346,173","$25,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$220,000","$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$205,200","$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$190,100","$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$184,100","$319,203"


## Clean and Prep Data

In [100]:
nba_sal.columns

Index(['Player', '2021/22', '2021/22(*)', '2020/21', '2020/21(*)', '2019/20',
       '2019/20(*)', '2018/19', '2018/19(*)', '2017/18', '2017/18(*)',
       '2016/17', '2016/17(*)', '2015/16', '2015/16(*)', '2014/15',
       '2014/15(*)', '2013/14', '2013/14(*)', '2012/13', '2012/13(*)',
       '2011/12', '2011/12(*)', '2010/11', '2010/11(*)', '2009/10',
       '2009/10(*)', '2008/09', '2008/09(*)', '2007/08', '2007/08(*)',
       '2006/07', '2006/07(*)', '2005/06', '2005/06(*)', '2004/05',
       '2004/05(*)', '2003/04', '2003/04(*)', '2002/03', '2002/03(*)',
       '2001/02', '2001/02(*)', '2000/01', '2000/01(*)', '1999/00',
       '1999/00(*)', '1998/99', '1998/99(*)', '1997/98', '1997/98(*)',
       '1996/97', '1996/97(*)'],
      dtype='object')

In [101]:
# Keep only columns that show current US Dollars adjusted for inflation, those columns with *
nba_sal = nba_sal.drop(['2021/22','2020/21', '2019/20', '2018/19', '2017/18',
                       '2016/17', '2015/16', '2014/15', '2013/14', '2012/13',
                       '2011/12', '2010/11', '2009/10', '2008/09', '2007/08',
                       '2006/07', '2005/06', '2004/05', '2003/04', '2002/03',
                       '2001/02', '2000/01', '1999/00', '1998/99', '1997/98',
                       '1996/97'], axis=1)

In [102]:
nba_sal

,Player,2021/22(*),2020/21(*),2019/20(*),2018/19(*),2017/18(*),2016/17(*),2015/16(*),2014/15(*),2013/14(*),...,2005/06(*),2004/05(*),2003/04(*),2002/03(*),2001/02(*),2000/01(*),1999/00(*),1998/99(*),1997/98(*),1996/97(*)
0,Stephen Curry,"$45,780,966","$45,325,029","$42,674,629","$40,386,520","$38,468,739","$13,654,081","$12,945,956","$12,116,632","$11,504,868",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$43,479,158","$40,518,442","$32,960,751","$31,388,759","$29,918,259","$17,896,589","$16,750,126","$15,904,415",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$43,479,158","$40,518,442","$20,668,989","$20,035,826","$17,761,997","$16,769,198","$15,618,561","$8,680,071",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$43,588,654","$40,844,595","$38,454,209","$31,388,759","$29,918,259","$19,063,757","$17,918,740","$17,097,246",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,271,404","$39,457,722","$32,346,173","$27,729,174","$29,918,259","$25,015,562","$22,795,904","$21,843,723",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$319,203"


In [103]:
# Clean up the column names
nba_sal = nba_sal.rename(columns = {'2021/22(*)':'2021-22','2020/21(*)':'2020-21', 
                                    '2019/20(*)':'2019-20', '2018/19(*)':'2018-19', 
                                    '2017/18(*)':'2017-18','2016/17(*)':'2016-17', 
                                    '2015/16(*)':'2015-16', '2014/15(*)':'2014-15', 
                                    '2013/14(*)':'2013-14', '2012/13(*)':'2012-13',
                                    '2011/12(*)':'2011-12', '2010/11(*)':'2010-11', 
                                    '2009/10(*)':'2009-10', '2008/09(*)':'2008-09', 
                                    '2007/08(*)':'2007-08','2006/07(*)':'2006-07', 
                                    '2005/06(*)':'2005-06', '2004/05(*)':'2004-05', 
                                    '2003/04(*)':'2003-04', '2002/03(*)':'2002-03',
                                    '2001/02(*)':'2001-02', '2000/01(*)':'2000-01', 
                                    '1999/00(*)':'1999-00', '1998/99(*)':'1998-99', 
                                    '1997/98(*)':'1997-98','1996/97(*)':'1996-97'})

nba_sal

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,"$45,780,966","$45,325,029","$42,674,629","$40,386,520","$38,468,739","$13,654,081","$12,945,956","$12,116,632","$11,504,868",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$43,479,158","$40,518,442","$32,960,751","$31,388,759","$29,918,259","$17,896,589","$16,750,126","$15,904,415",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$43,479,158","$40,518,442","$20,668,989","$20,035,826","$17,761,997","$16,769,198","$15,618,561","$8,680,071",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$43,588,654","$40,844,595","$38,454,209","$31,388,759","$29,918,259","$19,063,757","$17,918,740","$17,097,246",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,271,404","$39,457,722","$32,346,173","$27,729,174","$29,918,259","$25,015,562","$22,795,904","$21,843,723",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$319,203"


In [104]:
# Remove all '$' signs
nba_sal[nba_sal.columns] = nba_sal[nba_sal.columns].replace({'\$':''}, regex = True)
nba_sal

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,"45,780,966","45,325,029","42,674,629","40,386,520","38,468,739","13,654,081","12,945,956","12,116,632","11,504,868",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"44,310,840","43,479,158","40,518,442","32,960,751","31,388,759","29,918,259","17,896,589","16,750,126","15,904,415",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"44,310,840","43,479,158","40,518,442","20,668,989","20,035,826","17,761,997","16,769,198","15,618,561","8,680,071",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"44,211,146","43,588,654","40,844,595","38,454,209","31,388,759","29,918,259","19,063,757","17,918,740","17,097,246",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"42,018,900","42,271,404","39,457,722","32,346,173","27,729,174","29,918,259","25,015,562","22,795,904","21,843,723",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"319,203"


In [105]:
# Remove all commas ','
nba_sal = nba_sal.replace(',', '', regex=True)
nba_sal

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,45780966,45325029,42674629,40386520,38468739,13654081,12945956,12116632,11504868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,44310840,43479158,40518442,32960751,31388759,29918259,17896589,16750126,15904415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,44310840,43479158,40518442,20668989,20035826,17761997,16769198,15618561,8680071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,44211146,43588654,40844595,38454209,31388759,29918259,19063757,17918740,17097246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,42018900,42271404,39457722,32346173,27729174,29918259,25015562,22795904,21843723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381449
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355788
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329606
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319203


In [106]:
nba_sal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2491 entries, 0 to 2490
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   2491 non-null   object
 1   2021-22  653 non-null    object
 2   2020-21  578 non-null    object
 3   2019-20  515 non-null    object
 4   2018-19  576 non-null    object
 5   2017-18  586 non-null    object
 6   2016-17  545 non-null    object
 7   2015-16  500 non-null    object
 8   2014-15  514 non-null    object
 9   2013-14  492 non-null    object
 10  2012-13  494 non-null    object
 11  2011-12  463 non-null    object
 12  2010-11  459 non-null    object
 13  2009-10  456 non-null    object
 14  2008-09  460 non-null    object
 15  2007-08  469 non-null    object
 16  2006-07  495 non-null    object
 17  2005-06  479 non-null    object
 18  2004-05  470 non-null    object
 19  2003-04  454 non-null    object
 20  2002-03  451 non-null    object
 21  2001-02  450 non-null    object
 22  

In [107]:
# Get column names
nba_sal.columns

Index(['Player', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18',
       '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12',
       '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06',
       '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00',
       '1998-99', '1997-98', '1996-97'],
      dtype='object')

In [108]:
# Change datatypes to numeric for appropriate columns
nba_sal[['2021-22', '2020-21', '2019-20', '2018-19', '2017-18',
       '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12',
       '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06',
       '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00',
       '1998-99', '1997-98', '1996-97']] = nba_sal[['2021-22', '2020-21', '2019-20', '2018-19', '2017-18',
       '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12',
       '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06',
       '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00',
       '1998-99', '1997-98', '1996-97']].apply(pd.to_numeric)

In [109]:
nba_sal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2491 entries, 0 to 2490
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   2491 non-null   object 
 1   2021-22  653 non-null    float64
 2   2020-21  578 non-null    float64
 3   2019-20  515 non-null    float64
 4   2018-19  576 non-null    float64
 5   2017-18  586 non-null    float64
 6   2016-17  545 non-null    float64
 7   2015-16  500 non-null    float64
 8   2014-15  514 non-null    float64
 9   2013-14  492 non-null    float64
 10  2012-13  494 non-null    float64
 11  2011-12  463 non-null    float64
 12  2010-11  459 non-null    float64
 13  2009-10  456 non-null    float64
 14  2008-09  460 non-null    float64
 15  2007-08  469 non-null    float64
 16  2006-07  495 non-null    float64
 17  2005-06  479 non-null    float64
 18  2004-05  470 non-null    float64
 19  2003-04  454 non-null    float64
 20  2002-03  451 non-null    float64
 21  2001-02  450 n

In [110]:
# Find the average salary for each player
# Format average salary float to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format
nba_avg_sal = nba_sal.mean(axis=1)
nba_avg_sal

C:\Users\sandr\AppData\Local\Temp/ipykernel_28852/3382875248.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  nba_avg_sal = nba_sal.mean(axis=1)


0      21410995.23
1      22771817.00
2      20659679.75
3      22775948.79
4      23162299.73
           ...    
2486     381449.00
2487     355788.00
2488     329606.00
2489     319203.00
2490     265107.00
Length: 2491, dtype: float64

In [111]:
# Add average salary column in dataframe
nba_sal['AVG SALARY'] = nba_sal.mean(axis=1)
nba_sal.sort_values(by='Player', ascending=True).tail(20)

C:\Users\sandr\AppData\Local\Temp/ipykernel_28852/1617413931.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  nba_sal['AVG SALARY'] = nba_sal.mean(axis=1)


,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97,AVG SALARY
1136,Zach Auguste,NaN,NaN,NaN,NaN,NaN,67637.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67637.00
174,Zach Collins,7000000.00,5697730.00,4497664.00,3912722.00,3390989.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4899821.00
58,Zach LaVine,19500000.00,20551333.00,20684039.00,21025012.00,3551793.00,2526110.00,2445967.00,2343528.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11578472.75
894,Zach Lofton,NaN,NaN,NaN,83291.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83291.00
795,Zach Randolph,NaN,NaN,NaN,12606714.00,13651285.00,11272850.00,11385278.00,18808960.00,20711374.00,...,2154479.00,1733648.00,1654823.00,1555810.00,NaN,NaN,NaN,NaN,NaN,13450542.11
2189,Zan Tabak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1891155.00,1798228.00,NaN,1610175.00,1387088.00,1671661.50
1769,Zarko Cabarkapa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1880245.00,1806238.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2134178.75
650,Zavier Simpson,37223.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37223.00
828,Zaza Pachulia,NaN,NaN,NaN,2581102.00,3857239.00,3266872.00,5920344.00,5927672.00,6050513.00,...,888054.00,542698.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4585967.12
315,Zeke Nnaji,2498760.00,2508147.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2503453.50


In [112]:
# We only need 2 columns, the player and avg salary columns
nba_salary = nba_sal[['Player', 'AVG SALARY']]
nba_salary

,Player,AVG SALARY
0,Stephen Curry,21410995.23
1,James Harden,22771817.00
2,John Wall,20659679.75
3,Russell Westbrook,22775948.79
4,Kevin Durant,23162299.73
...,...,...
2486,Brett Szabo,381449.00
2487,Earl Cureton,355788.00
2488,Kenny Smith,329606.00
2489,Lloyd Daniels,319203.00


In [113]:
# Save to csv file
nba_salary.to_csv('nba_salaries_cleaned.csv', encoding='utf-8', index=False)

In [148]:
# Import nba_stats
nba_stats = pd.read_csv('nba_stats_cleaned.csv')
nba_stats

,PLAYER,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,A.C. Green,35.00,75.80,35.80,40.00,1881.00,444.60,180.20,397.00,44.98,...,83.80,123.40,67.00,158.00,312.20,470.20,67.20,51.80,51.80,15.40
1,AJ Price,25.50,43.50,19.83,23.67,654.67,253.83,91.50,240.67,38.00,...,34.50,46.50,62.07,11.33,51.50,62.83,94.50,38.17,20.83,1.50
2,Aaron Brooks,28.10,64.50,35.90,28.60,1343.90,625.90,226.10,547.80,41.09,...,91.70,109.50,82.00,25.40,82.50,107.90,192.50,100.00,35.90,8.70
3,Aaron Gordon,22.50,66.00,29.88,36.12,1909.50,860.88,325.62,707.88,45.96,...,136.25,194.00,70.06,105.12,308.62,413.75,164.12,99.00,48.62,41.88
4,Aaron Gray,26.00,45.43,17.71,27.71,550.00,152.29,62.86,123.43,50.86,...,26.57,47.29,56.59,61.57,108.43,170.00,29.57,37.86,13.43,12.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,Zach Randolph,28.00,65.65,33.00,32.65,2036.65,1092.82,437.41,927.88,47.04,...,206.29,270.12,75.32,192.82,407.59,600.41,120.53,134.71,49.06,17.71
1009,Zaza Pachulia,27.50,68.62,34.06,34.56,1389.56,463.38,163.00,347.44,46.91,...,137.38,182.94,74.97,147.62,247.06,394.69,89.56,90.25,48.00,23.62
1010,Zeljko Rebraca,32.00,43.00,24.00,19.00,655.60,255.20,97.60,185.20,52.18,...,60.00,75.80,78.90,39.20,98.40,137.60,17.60,40.60,11.40,32.00
1011,Zendon Hamilton,28.50,22.33,8.50,13.83,259.33,97.50,29.33,66.67,40.38,...,38.83,58.83,70.77,30.67,46.67,77.33,5.50,18.00,6.50,4.67


In [149]:
# Merge the two tables, nba_stats and nba_salary
nba_stats_sal = pd.merge(nba_salary, nba_stats.set_index('PLAYER'), left_on="Player", right_index=True)
nba_stats_sal

,Player,AVG SALARY,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,Stephen Curry,21410995.23,28.00,63.54,41.92,21.61,2181.69,1543.38,528.85,1116.92,...,245.92,270.77,90.84,42.54,252.69,295.23,414.46,199.85,105.15,14.38
1,James Harden,22771817.00,26.00,72.46,47.08,25.39,2509.08,1805.92,532.31,1203.92,...,541.85,630.38,85.61,57.08,350.15,407.23,492.08,272.23,111.46,39.38
2,John Wall,20659679.75,24.60,61.30,28.30,33.00,2183.20,1170.20,423.50,983.30,...,261.40,335.50,77.13,34.70,226.50,261.20,555.70,233.20,101.80,42.70
3,Russell Westbrook,22775948.79,26.50,72.93,42.71,30.21,2530.29,1664.14,594.43,1357.50,...,393.71,502.71,77.55,120.71,417.93,538.64,615.07,299.14,121.36,22.36
4,Kevin Durant,23162299.73,25.64,67.07,41.93,25.14,2464.93,1823.29,622.57,1255.50,...,451.71,510.93,88.19,47.07,427.64,474.71,288.00,211.71,72.93,74.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,Jerome Kersey,1230830.40,37.00,49.20,33.60,15.60,947.80,249.60,107.40,261.00,...,30.60,50.80,56.76,55.20,124.40,179.60,51.60,39.40,57.80,26.40
2208,Todd Fuller,1666712.60,24.00,45.00,19.80,25.20,499.60,166.20,63.00,150.00,...,40.20,60.20,71.66,48.00,86.80,134.80,9.20,29.20,6.80,12.00
2252,Scott Burrell,1966663.75,28.00,49.40,25.40,24.00,824.00,278.40,101.00,257.80,...,35.60,47.20,66.48,45.40,101.40,146.80,51.40,32.80,40.40,22.20
2282,Chucky Brown,894728.75,31.50,50.33,26.33,24.00,780.17,242.50,102.17,224.83,...,34.83,50.67,65.20,31.17,95.17,126.33,37.17,27.67,13.17,13.83


In [150]:
# Change column 'Player' for aesthetics
nba_stats_sal = nba_stats_sal.rename(columns = {'Player': 'PLAYER'})

nba_stats_sal

,PLAYER,AVG SALARY,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,Stephen Curry,21410995.23,28.00,63.54,41.92,21.61,2181.69,1543.38,528.85,1116.92,...,245.92,270.77,90.84,42.54,252.69,295.23,414.46,199.85,105.15,14.38
1,James Harden,22771817.00,26.00,72.46,47.08,25.39,2509.08,1805.92,532.31,1203.92,...,541.85,630.38,85.61,57.08,350.15,407.23,492.08,272.23,111.46,39.38
2,John Wall,20659679.75,24.60,61.30,28.30,33.00,2183.20,1170.20,423.50,983.30,...,261.40,335.50,77.13,34.70,226.50,261.20,555.70,233.20,101.80,42.70
3,Russell Westbrook,22775948.79,26.50,72.93,42.71,30.21,2530.29,1664.14,594.43,1357.50,...,393.71,502.71,77.55,120.71,417.93,538.64,615.07,299.14,121.36,22.36
4,Kevin Durant,23162299.73,25.64,67.07,41.93,25.14,2464.93,1823.29,622.57,1255.50,...,451.71,510.93,88.19,47.07,427.64,474.71,288.00,211.71,72.93,74.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,Jerome Kersey,1230830.40,37.00,49.20,33.60,15.60,947.80,249.60,107.40,261.00,...,30.60,50.80,56.76,55.20,124.40,179.60,51.60,39.40,57.80,26.40
2208,Todd Fuller,1666712.60,24.00,45.00,19.80,25.20,499.60,166.20,63.00,150.00,...,40.20,60.20,71.66,48.00,86.80,134.80,9.20,29.20,6.80,12.00
2252,Scott Burrell,1966663.75,28.00,49.40,25.40,24.00,824.00,278.40,101.00,257.80,...,35.60,47.20,66.48,45.40,101.40,146.80,51.40,32.80,40.40,22.20
2282,Chucky Brown,894728.75,31.50,50.33,26.33,24.00,780.17,242.50,102.17,224.83,...,34.83,50.67,65.20,31.17,95.17,126.33,37.17,27.67,13.17,13.83


In [151]:
# Save into csv file
nba_stats_sal.to_csv('nba_stats_merged.csv', encoding='utf-8', index=False)

In [152]:
# Read file
nba_complete = pd.read_csv('nba_stats_merged.csv', encoding='utf-8')
nba_complete

,PLAYER,AVG SALARY,AGE,GP,W,L,MIN,PTS,FGM,FGA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK
0,Stephen Curry,21410995.23,28.00,63.54,41.92,21.61,2181.69,1543.38,528.85,1116.92,...,245.92,270.77,90.84,42.54,252.69,295.23,414.46,199.85,105.15,14.38
1,James Harden,22771817.00,26.00,72.46,47.08,25.39,2509.08,1805.92,532.31,1203.92,...,541.85,630.38,85.61,57.08,350.15,407.23,492.08,272.23,111.46,39.38
2,John Wall,20659679.75,24.60,61.30,28.30,33.00,2183.20,1170.20,423.50,983.30,...,261.40,335.50,77.13,34.70,226.50,261.20,555.70,233.20,101.80,42.70
3,Russell Westbrook,22775948.79,26.50,72.93,42.71,30.21,2530.29,1664.14,594.43,1357.50,...,393.71,502.71,77.55,120.71,417.93,538.64,615.07,299.14,121.36,22.36
4,Kevin Durant,23162299.73,25.64,67.07,41.93,25.14,2464.93,1823.29,622.57,1255.50,...,451.71,510.93,88.19,47.07,427.64,474.71,288.00,211.71,72.93,74.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,Jerome Kersey,1230830.40,37.00,49.20,33.60,15.60,947.80,249.60,107.40,261.00,...,30.60,50.80,56.76,55.20,124.40,179.60,51.60,39.40,57.80,26.40
954,Todd Fuller,1666712.60,24.00,45.00,19.80,25.20,499.60,166.20,63.00,150.00,...,40.20,60.20,71.66,48.00,86.80,134.80,9.20,29.20,6.80,12.00
955,Scott Burrell,1966663.75,28.00,49.40,25.40,24.00,824.00,278.40,101.00,257.80,...,35.60,47.20,66.48,45.40,101.40,146.80,51.40,32.80,40.40,22.20
956,Chucky Brown,894728.75,31.50,50.33,26.33,24.00,780.17,242.50,102.17,224.83,...,34.83,50.67,65.20,31.17,95.17,126.33,37.17,27.67,13.17,13.83
